In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.Builder().appName("lets_test").config("spark.driver.memory", "16g").enableHiveSupport().getOrCreate()

24/10/31 00:23:30 WARN Utils: Your hostname, LostVayne resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface enp92s0)
24/10/31 00:23:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/31 00:23:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.csv("vgsales.csv", header=True)
df = df.select(*[F.col(x).alias(x.lower()) for x in df.columns])
df.show()

+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|rank|                name|platform|year|       genre|           publisher|na_sales|eu_sales|jp_sales|other_sales|global_sales|
+----+--------------------+--------+----+------------+--------------------+--------+--------+--------+-----------+------------+
|   1|          Wii Sports|     Wii|2006|      Sports|            Nintendo|   41.49|   29.02|    3.77|       8.46|       82.74|
|   2|   Super Mario Bros.|     NES|1985|    Platform|            Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|
|   3|      Mario Kart Wii|     Wii|2008|      Racing|            Nintendo|   15.85|   12.88|    3.79|       3.31|       35.82|
|   4|   Wii Sports Resort|     Wii|2009|      Sports|            Nintendo|   15.75|   11.01|    3.28|       2.96|          33|
|   5|Pokemon Red/Pokem...|      GB|1996|Role-Playing|            Nintendo|   11.27|    8.89|   10.22|  

In [3]:
df.repartition(12).write.mode("overwrite").partitionBy("genre", "year").option("path", "hdfs://localhost:9000/dev/sales/test").saveAsTable("dev.sales_test")

24/10/31 00:23:35 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/10/31 00:23:35 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/10/31 00:23:35 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/10/31 00:23:35 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/10/31 00:23:35 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/10/31 00:23:35 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/10/31 00:23:36 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
24/10/31 00:25:30 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
24/10/31 00:25:30 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
24/10/31 00:25:30 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exis

In [4]:
df = spark.sql("select * from dev.sales_test")
df.show()

+-----+--------------------+--------+--------------------+--------+--------+--------+-----------+------------+------+----+
| rank|                name|platform|           publisher|na_sales|eu_sales|jp_sales|other_sales|global_sales| genre|year|
+-----+--------------------+--------+--------------------+--------+--------+--------+-----------+------------+------+----+
|10591|All Kamen Rider: ...|      DS|  Namco Bandai Games|       0|       0|     0.1|          0|         0.1|Action|2012|
|15825|Ore wa Shoujo Man...|     PSP|              Giza10|       0|       0|    0.02|          0|        0.02|Action|2012|
| 2279|Harvest Moon 3D: ...|     3DS|Marvelous Enterta...|    0.41|    0.17|    0.29|       0.05|        0.91|Action|2012|
| 5487|Medal of Honor: W...|      PC|     Electronic Arts|    0.14|    0.15|       0|       0.04|        0.33|Action|2012|
| 2975|Kirby's Dream Col...|     Wii|            Nintendo|    0.34|       0|    0.33|       0.02|        0.68|Action|2012|
|11139|Jikkyou P